# F@n Internship DL project 

### Multi-label classification

## 1. Preprocessing

In [129]:
# クリエイティブリストクローリング用
#from bs4 import BeautifulSoup
import requests
import pandas as pd
from tqdm import tqdm  # プログレスバー表示のため
from pathlib import Path # 画像ダウンロード&保存用 (全部やると死ぬので一部のみ)
from sklearn.utils import resample

# import yaml
# import pandas_td as td
import numpy as np
# import tdclient

parent_url = 'http://13.230.115.40/spool/creative/'

df = pd.read_csv('image_labels_multi.csv')

In [130]:
for index, x in df.iterrows():
    if '.gif' in df.at[index,'filename']:
        df.at[index,'filename'] = None

df = df.dropna()

In [2]:
import pandas as pd
df = pd.read_csv('image_labels_multi.csv')
df.query('extend!="gif"').describe().transpose()

,count,mean,std,min,25%,50%,75%,max
general,150634.0,8.226363e-01,0.381977,0.0,1.00,1.0,1.00,1.0
wedding,150634.0,8.032715e-04,0.028331,0.0,0.00,0.0,0.00,1.0
meet,150634.0,1.564056e-02,0.124081,0.0,0.00,0.0,0.00,1.0
live_chat,150634.0,2.648804e-03,0.051398,0.0,0.00,0.0,0.00,1.0
adult,150634.0,5.213962e-02,0.222309,0.0,0.00,0.0,0.00,1.0
loan,150634.0,8.278344e-03,0.090608,0.0,0.00,0.0,0.00,1.0
complex,150634.0,9.499847e-03,0.097003,0.0,0.00,0.0,0.00,1.0
beauty,150634.0,3.994450e-02,0.195829,0.0,0.00,0.0,0.00,1.0
present,150634.0,2.655443e-05,0.005153,0.0,0.00,0.0,0.00,1.0
gambling,150634.0,6.904152e-04,0.026267,0.0,0.00,0.0,0.00,1.0


In [131]:
labels = df.iloc[:,1:19]
labels["total_labels"] = labels.sum(axis=1, numeric_only = True)

for index, x in labels.iterrows():
    if labels.at[index, 'total_labels'] == 0:
        labels.at[index,'general'] = 1

In [132]:
labels["total_labels"] = labels.iloc[:,:-1].sum(axis=1, numeric_only = True)

In [133]:
file_n = df.loc[:,'filename']
label_name = df.loc[:,'label_name']

In [134]:
columns = labels.columns.tolist()
columns = columns[:-1]
labels['path'] = None

labels = pd.concat([labels,file_n],axis=1)
labels = pd.concat([labels,label_name],axis=1)

# insert the path of the images in path column
for index, x in labels.iterrows():
    labels.at[index,'path'] = x['label_name'] + "/" + x['filename']

In [135]:
only_general = labels[((labels['general']==1)&(labels['total_labels']==1))]
print(len(only_general))
print(len(labels))

print(len(only_general)/len(labels))

124136
150634
0.8240901788440856


In [151]:
# Downsample not_adult class
not_general = labels[~((labels['general']==1)&(labels['total_labels']==1))]

N = 1 # downsample ratio

df_only_general_downsampled = resample(only_general,
                                   replace=False,    # sample without replacement
                                   n_samples=not_general*N, # to match minority class
                                      )#random_state=43) # reproducible results
df_downsampled = pd.concat([df_only_general_downsampled, df_not_general])

TypeError: '>' not supported between instances of 'str' and 'int'

In [150]:
not_general.shape

(26498, 22)

In [136]:
# import keras modules
from keras.models import Sequential
"""Import from keras_preprocessing not from keras.preprocessing, because Keras may or maynot contain the features discussed here depending upon when you read this article, until the keras_preprocessed library is updated in Keras use the github version."""
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization, Input
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
from keras.applications.inception_v3 import InceptionV3
from keras.models import Model, load_model
from keras.layers.core import Dense
from keras.layers.pooling import GlobalAveragePooling2D
from keras.optimizers import Adam, RMSprop, SGD
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
import keras.backend as K

import numpy as np

In [137]:
from sklearn.model_selection import train_test_split

# split train/test data
traindf, testdf = train_test_split(labels, test_size = 0.1)

In [138]:
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.25)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator=datagen.flow_from_dataframe(
    dataframe = traindf,
    directory = "./dataset",
    x_col = "path",
    y_col = columns,
    batch_size = 32,
    seed = 42,
    shuffle = True,
    class_mode = 'other',
    subset = 'training', #If you use validation_split don't forget
    target_size = (200,200)
)

valid_generator=datagen.flow_from_dataframe(
    dataframe = traindf,
    directory = "./dataset",
    x_col = "path",
    y_col = columns,
    batch_size = 32,
    seed = 42,
    shuffle = True,
    class_mode = 'other',
    subset = 'validation',
    target_size = (200,200)
)

test_generator=test_datagen.flow_from_dataframe(
    dataframe = testdf,
    directory = "./dataset",
    x_col = "path",
    batch_size = 1,
    seed = 42,
    shuffle = False,
    class_mode = None,
    target_size = (200,200)
)

Found 101678 images.
Found 33892 images.
Found 15064 images.


## 2. Build Model 

In [139]:
# hyper parameters
img_size = 200
nb_train_samples = train_generator.samples
nb_validation_samples = valid_generator.samples
batch_size = 32
nb_epochs = 10

#build the model
input_tensor = Input(shape=(img_size, img_size, 3))
base_model = InceptionV3(include_top = False, weights = "imagenet",
                        input_tensor = input_tensor)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(len(columns), activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 200, 200, 3)  0                                            
__________________________________________________________________________________________________
conv2d_95 (Conv2D)              (None, 99, 99, 32)   864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_95 (BatchNo (None, 99, 99, 32)   96          conv2d_95[0][0]                  
__________________________________________________________________________________________________
activation_95 (Activation)      (None, 99, 99, 32)   0           batch_normalization_95[0][0]     
__________________________________________________________________________________________________
conv2d_96 

In [140]:
# freeze all weights remaining only the top layer
for layer in base_model.layers:
    layer.trainable == False

# compile model
model.compile(optimizer = 'Adam', loss='binary_crossentropy', metrics=['accuracy'])

## 3. Train the compiled model

In [141]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size

checkpointer = ModelCheckpoint(filepath='/home/c_kamiya/dl_project/models/multi_label_image.h5', verbose=1, save_best_only=True)

model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=1,
                    callbacks=[checkpointer]
)

Epoch 1/1


/home/c_kamiya/.conda/envs/keras-gpu/lib/python3.6/site-packages/PIL/Image.py:969: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


 622/3177 [====>.........................] - ETA: 24:47 - loss: 0.0801 - acc: 0.9798

KeyboardInterrupt: 

## 4. Predict

In [13]:
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score

#Model evaluation with test data
#load model
model_to_eval = load_model("/home/c_kamiya/dl_project/models/multi_label_image.h5")

test_generator.reset()
test_pred = model_to_eval.predict_generator(test_generator, steps = test_generator.samples // test_generator.batch_size, verbose = 1)


  392/15064 [..............................] - ETA: 10:50

/home/c_kamiya/.conda/envs/keras-gpu/lib/python3.6/site-packages/PIL/Image.py:969: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


15064/15064 [==============================] - 529s 35ms/step


In [96]:

predicted_class_indices = test_pred.argmax(axis=1)
pred_bool = (test_pred >0.5)

predictions = pred_bool.astype(int)

results=pd.DataFrame(predictions, columns=columns)
results["Filenames"]=test_generator.filenames
ordered_cols=["Filenames"]+columns
results=results[ordered_cols]

In [124]:
test_true = testdf.iloc[:,0:19]


In [125]:
test_true

,general,wedding,meet,live_chat,adult,loan,complex,beauty,present,gambling,sexual,comunity,ebook_adult_strong,ebook_adult_weak,other_deliver,game_adult_strong,game_adult_weak,politics,total_labels
228517,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
10574,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
177082,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
227553,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
9752,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
25891,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
157006,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
62810,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
18836,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
244512,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [ ]:
# cm = confusion_matrix(test_true, pred_bool) #confusion matrix
# acc = accuracy_score(test_true, pred_bool) #accuracy
f_score = f1_score(test_true, pred_bool) #f-1 score

In [113]:
# print("Confusion Matrix:")
# print(cm)
# print("Accuracy:")
# print(acc)
print("F_score",average='macro')
print(f_score)

ValueError: Multi-label binary indicator input with different numbers of labels

In [94]:
def show_example(idx):
    N_true = int(np.sum(testdf[idx]))
    show_img(ids[n + idx])
    print("Prediction: {}".format("|".join(["{} ({:.3})".format(label_dict["idx2word"][s],
                                                                pred[idx][s])
                                            for s in pred[idx].argsort()[-N_true:][::-1]])))

In [97]:
results

,Filenames,general,wedding,meet,live_chat,adult,loan,complex,beauty,present,gambling,sexual,comunity,ebook_adult_strong,ebook_adult_weak,other_deliver,game_adult_strong,game_adult_weak,politics,total_labels
0,not_adult/3129667_f4f6ba41ab20a9c2608f03483d42...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,not_adult/3211999_6d20df626998c209e8906726707e...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,not_adult/3015408_f9177b29c5a78ba9b0e94e978bbe...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,adult/3127035_2481ecaa3b75144326f5066c611d35a4...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,not_adult/3217170_1286c664164fbe9ff24b016f0f5e...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
5,not_adult/3245574_b286affa46de7a9c99ef8858dbce...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
6,not_adult/2954152_4e5051236b3fdc0b9900c7ddb7a2...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
7,not_adult/3331502_775a36c4d7fb4848d80f668fc21c...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
8,adult/3235749_4cf8239c0e67534a3496ab74b35bb435...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
9,not_adult/3158239_7f4dde20bf9adf2fd82e09ab1236...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [1]:
columns

NameError: name 'columns' is not defined